In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline

In [2]:
data = pd.read_csv("NY-House-Dataset.csv")
data.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856


In [3]:
data.describe()

,PRICE,BEDS,BATH,PROPERTYSQFT,LATITUDE,LONGITUDE
count,4.801000e+03,4801.000000,4801.000000,4801.000000,4801.000000,4801.000000
mean,2.356940e+06,3.356801,2.373861,2184.207862,40.714227,-73.941601
std,3.135525e+07,2.602315,1.946962,2377.140894,0.087676,0.101082
min,2.494000e+03,1.000000,0.000000,230.000000,40.499546,-74.253033
25%,4.990000e+05,2.000000,1.000000,1200.000000,40.639375,-73.987143
50%,8.250000e+05,3.000000,2.000000,2184.207862,40.726749,-73.949189
75%,1.495000e+06,4.000000,3.000000,2184.207862,40.771923,-73.870638
max,2.147484e+09,50.000000,50.000000,65535.000000,40.912729,-73.702450


In [4]:
data["STATE"].value_counts()

Brooklyn, NY 11235            102
Brooklyn, NY 11209             94
Forest Hills, NY 11375         91
Brooklyn, NY 11234             90
Staten Island, NY 10314        86
                             ... 
Queens, NY 11427                1
Queens, NY 11429                1
Long Island City, NY 11109      1
Brooklyn, NY 11694              1
Ozone Park, NY 11416            1
Name: STATE, Length: 308, dtype: int64

In [5]:
city = []
for i in data["STATE"]:
    city.append(i.split(",")[0])
data["AREA"] = pd.Series(city)
data["AREA"]

0            New York
1            New York
2       Staten Island
3           Manhattan
4            New York
            ...      
4796        Manhattan
4797        Rego Park
4798         New York
4799         Elmhurst
4800         Brooklyn
Name: AREA, Length: 4801, dtype: object

In [6]:
data["LOCALITY"].value_counts()

New York           2505
New York County     979
Queens County       557
Kings County        464
Bronx County        182
Richmond County      59
United States        37
Brooklyn              6
Queens                6
The Bronx             5
Flatbush              1
Name: LOCALITY, dtype: int64

In [7]:
data["SUBLOCALITY"].value_counts()

New York            1016
Kings County         727
Queens County        684
Queens               555
Richmond County      480
Brooklyn             464
Bronx County         307
New York County      286
The Bronx            184
Staten Island         59
Manhattan             21
Riverdale              4
Flushing               4
Coney Island           3
East Bronx             1
Brooklyn Heights       1
Jackson Heights        1
Rego Park              1
Fort Hamilton          1
Dumbo                  1
Snyder Avenue          1
Name: SUBLOCALITY, dtype: int64

In [8]:
data.columns

Index(['BROKERTITLE', 'TYPE', 'PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT',
       'ADDRESS', 'STATE', 'MAIN_ADDRESS', 'ADMINISTRATIVE_AREA_LEVEL_2',
       'LOCALITY', 'SUBLOCALITY', 'STREET_NAME', 'LONG_NAME',
       'FORMATTED_ADDRESS', 'LATITUDE', 'LONGITUDE', 'AREA'],
      dtype='object')

In [30]:
def preprocessData(data):
    dataX = data[["BEDS","BATH","PROPERTYSQFT","LATITUDE","LONGITUDE"]]
    dataY = data[["PRICE"]]
    x_train, x_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.2)
    print("Data split into train and test sets...")
    scaler = StandardScaler()
    scaler = MinMaxScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    print(x_train_scaled.shape)
    print(x_test_scaled.shape)

    print("Data Normalized...")
    return [x_train_scaled, y_train, x_test_scaled, y_test]


In [31]:
def modelBuilding(data):
    trainX, trainY, testX, testY = data
    lr = LinearRegression()
    lr.fit(trainX, trainY)
    print("Data fit to model...")
    print("Coefficient of Determination/R-squared: ", lr.score(trainX, trainY))
    preds = lr.predict(testX)
    print("Model perform prediction...")
#     print("Coefficients of prediction: ",lr.coef_)
    return [preds, testY]

In [32]:
def evaluate(results):
    preds, testY = results
    print("Root Mean Squared Error: ",np.sqrt(mean_squared_error(testY, preds)))
    print("Coefficient of Determination/R-squared: ", r2_score(testY, preds))
    print(testY, preds)

In [33]:
evaluate(modelBuilding(preprocessData(data)))

Data split into train and test sets...
(3840, 5)
(961, 5)
Data Normalized...
Data fit to model...
Coefficient of Determination/R-squared:  0.014319131706481247
Model perform prediction...
Root Mean Squared Error:  6976673.032937604
Coefficient of Determination/R-squared:  0.12226126686275074
         PRICE
622    3000000
2222   1195000
4581    890000
4012   2795000
1877  12495000
...        ...
1352    979000
4064   2975000
2683   4750000
2715   1450000
4016    780000

[961 rows x 1 columns] [[ 4.53160463e+07]
 [ 2.72316526e+06]
 [ 3.14148258e+06]
 [ 2.25715511e+06]
 [ 9.78925798e+06]
 [ 5.12992751e+06]
 [ 4.19300341e+06]
 [ 7.16545795e+05]
 [ 4.09905960e+05]
 [ 1.63636386e+07]
 [ 1.93116648e+06]
 [ 3.20727429e+06]
 [ 3.43126442e+06]
 [ 2.21682416e+06]
 [-3.84710093e+06]
 [-2.90359591e+05]
 [ 1.38895561e+06]
 [ 1.92941925e+05]
 [ 9.71465824e+06]
 [-1.60469837e+06]
 [ 3.12912559e+06]
 [ 3.11094943e+06]
 [-1.29311710e+06]
 [ 9.46150277e+05]
 [ 1.71825251e+06]
 [ 1.57221889e+06]
 [ 3.1332